This repository and project has a few personal and functional objectives

### Personal
1. Develop habit of using git and version control as I'm developing and writing functions
2. Teach myself how to write useful unit tests
3. Learn about flask and deploying an API

### Functional
1. Write an API that can return the number of days, weekdays and complete weeks between two arbitrary dates with
timezones and return the result in any specified time unit (seconds, minutes, hours, days, years)
2. Make the solution easy to deploy (possibly a docker container)
3. Once written in Python, reimplement in PHP (separate repo)

The purpose of this python notebook is to document the development process (outside of commits) and have small
tests of code as well design decisions

### The breakdown
It's always good to have a plan of how to tackle the problem and break it up into small chunks.

1. Core functionality. Finding days between two dates is easy, weekdays and complete weeks will need some thought
2. Write unit tests to validate the core functionality. Generate a set of dates and potential problem values,
check their results on some publicly available API (https://www.timeanddate.com/date/workdays.html), use these
values as correct behaviour
3. Write a parsing function to validate that the strings fed into the API generate valid aware datetime objects
4. Write unit tests to validate parsing function with a few inputs to make sure it gracefully handles results
5. Implement flask API and function routing
6. Write up some documentation

I expect I will get stuck on determining and algorithm for 1

In [6]:
# First we'll define some arbitrary strings to testing the function

time1 = '2020-09-14 00:00:00+08:30' # same local time in two different timezones
time2 = '2020-09-14 00:00:00+00:00' # same local time in two different timezones
time3 = '2016-02-29 00:00:00+00:00' # Leap year
time4 = '2017-11-23 13:00:50+00:00' # random date with a time

Let's check the web for what it thinks the values of these should be
1. time1-time2 is 8:30
2. time1-time3 is 1658 days, 14 hours, 30 minutes
3. time1-time4 is 1025 days, 1 hour, 29 minutes and 10 seconds

In [8]:
import datetime as dt
# Let's write the function to calculate the days between the dates
def days_between_dates(date1,date2):
    date1 = dt.datetime.fromisoformat(date1)
    date2 = dt.datetime.fromisoformat(date2)
    timedelta = date2-date1
    return timedelta

print(days_between_dates(time1, time2))
print(days_between_dates(time1, time3))
print(days_between_dates(time1, time4))

8:30:00
-1659 days, 8:30:00
-1026 days, 21:30:50


So some issues to resolve and debug, but the start is there. We've got the starting of our unit tests, as
well as something worth committing. We'll move this function to its own file now and just import into the note
book for testing.